# 📊 SOULFRIEND Research Data Collection Architecture
## Phân tích & Kế hoạch Triển khai Thu thập Dữ liệu Nghiên cứu Ẩn danh

**Ngày:** 27/08/2025  
**Dự án:** SOULFRIEND V2.0 Mental Health Assessment Platform  
**Mục tiêu:** Xây dựng hệ thống thu thập dữ liệu nghiên cứu tuân thủ quy định bảo mật và đạo đức

## 🎯 1. Tổng quan Kiến trúc Đề xuất

### Nguyên tắc Core
- **Consent-first**: Đồng ý có thông tin trước khi thu thập
- **Privacy by Design**: Ẩn danh từ giai đoạn thiết kế
- **Minimal Data**: Chỉ thu thập dữ liệu cần thiết cho nghiên cứu
- **Transparent**: Minh bạch quy trình với người dùng
- **Compliant**: Tuân thủ Nghị định 13/2023/NĐ-CP Việt Nam

### Kiến trúc High-level
```
Client (Streamlit) 
    ↓ Background POST
API Thu thập (FastAPI)
    ↓ Queue Jobs
Worker Xử lý (RQ/Celery)
    ↓ Anonymize
Research Database
    ↓ ETL Jobs
Dataset Nghiên cứu
```

In [ ]:
# Phân tích hiện trạng SOULFRIEND
import os
import json
from datetime import datetime

# Kiểm tra cấu trúc dự án hiện tại
project_structure = {
    "current_app": "SOULFRIEND.py",
    "questionnaires": ["PHQ-9", "GAD-7", "DASS-21", "EPDS", "PSS-10"],
    "components": [
        "questionnaires.py", "scoring.py", "ui.py", 
        "charts.py", "pdf_export.py", "admin_auth.py"
    ],
    "pages": ["Consent", "Assessment", "Results", "Resources", "Admin", "Chatbot"],
    "ready_features": [
        "Multi-questionnaire assessment",
        "PDF report generation", 
        "Admin panel",
        "Data encryption",
        "Mobile optimization"
    ]
}

print("🧠 SOULFRIEND Current State Analysis")
print("=" * 50)
for key, value in project_structure.items():
    print(f"📋 {key.title()}: {len(value) if isinstance(value, list) else value}")
    if isinstance(value, list) and len(value) <= 6:
        for item in value:
            print(f"   ✅ {item}")

## 🏗️ 2. Roadmap Triển khai

### Phase 1: Foundation (2-3 tuần)
- **Week 1**: Database Schema & FastAPI Setup
- **Week 2**: Consent UI & Basic Collection
- **Week 3**: Worker & Anonymization Logic

### Phase 2: Core Collection (2 tuần)
- **Week 4**: Integration với SOULFRIEND
- **Week 5**: Testing & Security Audit

### Phase 3: Analytics & Compliance (1-2 tuần)
- **Week 6**: ETL Jobs & Dataset Generation
- **Week 7**: Admin Dashboard & Monitoring

In [ ]:
# Định nghĩa các component cần phát triển
implementation_components = {
    "database": {
        "tables": ["raw_events", "pii_mapping", "research_sessions", "research_events"],
        "priority": "HIGH",
        "estimated_hours": 8
    },
    "fastapi_collector": {
        "endpoints": ["/collect", "/health", "/consent-status"],
        "priority": "HIGH", 
        "estimated_hours": 12
    },
    "consent_ui": {
        "components": ["consent_form", "privacy_toggle", "data_usage_info"],
        "priority": "HIGH",
        "estimated_hours": 6
    },
    "anonymization_worker": {
        "functions": ["pseudonymize", "normalize_events", "apply_k_anonymity"],
        "priority": "MEDIUM",
        "estimated_hours": 16
    },
    "etl_pipeline": {
        "jobs": ["daily_dataset_gen", "data_quality_check", "retention_cleanup"],
        "priority": "MEDIUM",
        "estimated_hours": 10
    },
    "admin_dashboard": {
        "features": ["collection_stats", "dataset_management", "consent_analytics"],
        "priority": "LOW",
        "estimated_hours": 8
    }
}

total_hours = sum(comp["estimated_hours"] for comp in implementation_components.values())
print(f"⏱️ Total Estimated Development Time: {total_hours} hours ({total_hours/40:.1f} weeks)")
print("\n📋 Component Breakdown:")
for name, details in implementation_components.items():
    print(f"\n🔧 {name.replace('_', ' ').title()}")
    print(f"   Priority: {details['priority']}")
    print(f"   Time: {details['estimated_hours']} hours")
    if "tables" in details:
        print(f"   Tables: {', '.join(details['tables'])}")
    elif "endpoints" in details:
        print(f"   Endpoints: {', '.join(details['endpoints'])}")
    elif "components" in details:
        print(f"   UI: {', '.join(details['components'])}")

## 🗄️ 3. Database Schema Chi tiết

### Raw Events (Temporary Storage)
```sql
CREATE TABLE raw_events (
  id BIGSERIAL PRIMARY KEY,
  received_at TIMESTAMPTZ DEFAULT NOW(),
  client_ts TIMESTAMPTZ,
  session_id TEXT NOT NULL,
  user_hash TEXT,
  event_name TEXT NOT NULL,
  payload JSONB,
  processed BOOLEAN DEFAULT FALSE,
  retention_until TIMESTAMPTZ DEFAULT (NOW() + INTERVAL '90 days')
);
```

### PII Mapping (Secure, Separate DB)
```sql
CREATE TABLE pii_mapping (
  id BIGSERIAL PRIMARY KEY,
  user_hash TEXT UNIQUE NOT NULL,
  user_pseudo_id TEXT UNIQUE NOT NULL,
  created_at TIMESTAMPTZ DEFAULT NOW(),
  last_accessed TIMESTAMPTZ,
  access_reason TEXT
);
```

In [ ]:
# Tạo SQL schema cho toàn bộ hệ thống
database_schema = {
    "raw_events": {
        "purpose": "Temporary storage for incoming events",
        "retention": "90 days",
        "fields": [
            "id (BIGSERIAL)",
            "received_at (TIMESTAMPTZ)", 
            "client_ts (TIMESTAMPTZ)",
            "session_id (TEXT)",
            "user_hash (TEXT)",
            "event_name (TEXT)",
            "payload (JSONB)",
            "processed (BOOLEAN)"
        ]
    },
    "research_sessions": {
        "purpose": "Anonymized session metadata",
        "retention": "5 years",
        "fields": [
            "id (BIGSERIAL)",
            "user_pseudo_id (TEXT)",
            "session_id (TEXT)",
            "age_band (TEXT)",
            "cohort_version (TEXT)",
            "started_at (TIMESTAMPTZ)",
            "completed_at (TIMESTAMPTZ)",
            "consent_shared (BOOLEAN)",
            "questionnaires_completed (TEXT[])"
        ]
    },
    "research_events": {
        "purpose": "Anonymized individual responses",
        "retention": "5 years", 
        "fields": [
            "id (BIGSERIAL)",
            "session_id (TEXT)",
            "user_pseudo_id (TEXT)",
            "event_name (TEXT)",
            "questionnaire_type (TEXT)",
            "item_id (TEXT)",
            "response_value (SMALLINT)",
            "response_time_ms (INTEGER)",
            "client_ts_rounded (TIMESTAMPTZ)"
        ]
    }
}

print("🗄️ Research Database Schema")
print("=" * 40)
for table_name, details in database_schema.items():
    print(f"\n📋 {table_name.upper()}")
    print(f"   Purpose: {details['purpose']}")
    print(f"   Retention: {details['retention']}")
    print(f"   Fields ({len(details['fields'])}):") 
    for field in details['fields']:
        print(f"     • {field}")

## 🔐 4. Security & Compliance Framework

### GDPR & Nghị định 13/2023/NĐ-CP Compliance
- **Lawful Basis**: Consent (Art. 6(1)(a)) + Legitimate Interest for research
- **Data Minimization**: Chỉ thu thập dữ liệu cần thiết
- **Purpose Limitation**: Chỉ sử dụng cho nghiên cứu sức khỏe tâm thần
- **Storage Limitation**: Raw data 90 ngày, research data 5 năm
- **Right to Withdraw**: Có thể rút đồng ý bất kỳ lúc nào

### Technical Safeguards
- **Encryption at Rest**: PostgreSQL TDE
- **Encryption in Transit**: HTTPS/TLS 1.3
- **Pseudonymization**: HMAC-SHA256 với secret riêng
- **Access Control**: Role-based với audit logging
- **Data Separation**: PII mapping ở database riêng

In [ ]:
# Định nghĩa security requirements
security_requirements = {
    "encryption": {
        "at_rest": ["PostgreSQL TDE", "Volume encryption", "Key rotation"],
        "in_transit": ["HTTPS/TLS 1.3", "Certificate pinning", "HSTS headers"],
        "application": ["HMAC user hashing", "Session encryption", "Secret management"]
    },
    "access_control": {
        "authentication": ["Admin 2FA", "API key rotation", "Session timeout"],
        "authorization": ["Role-based access", "Principle of least privilege", "Audit logging"],
        "monitoring": ["Failed login alerts", "Suspicious activity detection", "Access pattern analysis"]
    },
    "data_protection": {
        "pseudonymization": ["HMAC-SHA256 hashing", "Separate key storage", "Hash rotation"],
        "anonymization": ["K-anonymity (k≥10)", "Age band generalization", "Timestamp rounding"],
        "retention": ["Automated cleanup", "Retention policies", "Secure deletion"]
    },
    "compliance": {
        "vietnam_law": ["Nghị định 13/2023/NĐ-CP", "Data localization", "Consent management"],
        "international": ["GDPR principles", "ISO 27001", "Healthcare data standards"],
        "ethics": ["IRB consideration", "Research ethics", "Vulnerable population protection"]
    }
}

print("🔐 Security & Compliance Framework")
print("=" * 45)
for category, subcategories in security_requirements.items():
    print(f"\n🛡️ {category.replace('_', ' ').title()}")
    for subcat, items in subcategories.items():
        print(f"   📋 {subcat.replace('_', ' ').title()}:")
        for item in items:
            print(f"     ✅ {item}")

## 🔄 5. Data Flow & Processing Pipeline

### Event Collection Flow
1. **User Action** → Streamlit UI
2. **Background POST** → FastAPI `/collect` endpoint
3. **Raw Storage** → `raw_events` table
4. **Queue Job** → Redis + RQ Worker
5. **Anonymization** → `research_*` tables
6. **ETL Processing** → Dataset generation

### Event Types to Collect
- `session_started` - Bắt đầu đánh giá
- `consent_given` - Đồng ý chia sẻ dữ liệu
- `questionnaire_started` - Bắt đầu bộ câu hỏi
- `question_answered` - Trả lời câu hỏi cụ thể
- `questionnaire_completed` - Hoàn thành bộ câu hỏi
- `results_viewed` - Xem kết quả
- `report_downloaded` - Tải báo cáo
- `session_ended` - Kết thúc phiên

In [ ]:
# Mô phỏng data flow và event structure
import json
from datetime import datetime, timedelta
import uuid

# Định nghĩa event types và payload structure
event_types = {
    "session_started": {
        "payload_fields": ["user_agent", "screen_resolution", "locale", "referrer"],
        "research_value": "HIGH",
        "frequency": "Once per session"
    },
    "consent_given": {
        "payload_fields": ["consent_version", "sharing_enabled", "research_participation"],
        "research_value": "CRITICAL",
        "frequency": "Once per session"
    },
    "questionnaire_started": {
        "payload_fields": ["questionnaire_type", "version", "estimated_duration"],
        "research_value": "HIGH", 
        "frequency": "Per questionnaire"
    },
    "question_answered": {
        "payload_fields": ["item_id", "response_value", "response_time_ms", "previous_answer"],
        "research_value": "CRITICAL",
        "frequency": "Per question"
    },
    "results_viewed": {
        "payload_fields": ["total_score", "severity_level", "recommendations_shown"],
        "research_value": "MEDIUM",
        "frequency": "Per result view"
    }
}

# Tạo example event payload
example_event = {
    "client_ts": datetime.utcnow().isoformat(),
    "session_id": str(uuid.uuid4()),
    "user_uuid": str(uuid.uuid4()),
    "event_name": "question_answered",
    "payload": {
        "questionnaire_type": "PHQ-9",
        "item_id": "PHQ9_Q07",
        "response_value": 2,
        "response_time_ms": 3450,
        "age_band": "25-34",
        "previous_answer": 1,
        "question_text_hash": "abc123def"
    },
    "cohort_version": "v2.0"
}

print("📊 Event Types Analysis")
print("=" * 30)
for event_name, details in event_types.items():
    print(f"\n🎯 {event_name.upper()}")
    print(f"   Research Value: {details['research_value']}")
    print(f"   Frequency: {details['frequency']}")
    print(f"   Payload Fields: {', '.join(details['payload_fields'])}")

print("\n📋 Example Event Payload:")
print(json.dumps(example_event, indent=2, default=str))

## 🚀 6. Implementation Action Plan

### Immediate Next Steps (Week 1)
1. **Setup Development Environment**
   - Docker Compose với PostgreSQL, Redis, FastAPI
   - Environment variables cho secrets
   - Basic CI/CD pipeline

2. **Database Schema Implementation**
   - Tạo migration scripts
   - Setup connection pooling
   - Implement retention policies

3. **FastAPI Collection Service**
   - `/collect` endpoint với validation
   - Rate limiting & security headers
   - Health check endpoint

### Week 2-3: Core Integration
1. **Consent Management UI**
   - Update existing Consent page
   - Add privacy controls
   - Implement consent versioning

2. **Event Collection Integration**
   - Modify SOULFRIEND.py event handlers
   - Add background POST calls
   - Error handling & fallbacks

3. **Worker & Anonymization**
   - RQ worker setup
   - Pseudonymization logic
   - Data validation & cleanup

In [ ]:
# Tạo implementation checklist với priority và dependencies
implementation_tasks = {
    "week_1": {
        "setup_environment": {
            "tasks": [
                "Create Docker Compose setup",
                "Configure PostgreSQL with TDE", 
                "Setup Redis for job queue",
                "Environment variables management",
                "Basic security hardening"
            ],
            "estimated_hours": 8,
            "priority": "CRITICAL"
        },
        "database_schema": {
            "tasks": [
                "Create migration scripts",
                "Implement all table schemas",
                "Setup indexes and constraints",
                "Configure retention policies",
                "Test data flows"
            ],
            "estimated_hours": 6,
            "priority": "CRITICAL"
        },
        "fastapi_service": {
            "tasks": [
                "Create /collect endpoint",
                "Add request validation",
                "Implement rate limiting",
                "Security headers & CORS",
                "Health check endpoint"
            ],
            "estimated_hours": 10,
            "priority": "HIGH"
        }
    },
    "week_2": {
        "consent_ui": {
            "tasks": [
                "Update Consent page design",
                "Add privacy toggle controls",
                "Implement consent versioning",
                "Add data usage explanation",
                "Mobile optimization"
            ],
            "estimated_hours": 8,
            "priority": "HIGH"
        },
        "event_integration": {
            "tasks": [
                "Add event collection to SOULFRIEND",
                "Implement background POST calls",
                "Error handling & retries",
                "Session state management",
                "Testing event flows"
            ],
            "estimated_hours": 12,
            "priority": "HIGH"
        }
    },
    "week_3": {
        "worker_anonymization": {
            "tasks": [
                "Setup RQ worker processes",
                "Implement pseudonymization",
                "Data validation & cleaning",
                "K-anonymity checks",
                "Error handling & monitoring"
            ],
            "estimated_hours": 16,
            "priority": "MEDIUM"
        }
    }
}

# Calculate total effort
total_weeks = len(implementation_tasks)
total_tasks = sum(len(week[task_group]["tasks"]) for week in implementation_tasks.values() for task_group in week.keys())
total_hours = sum(week[task_group]["estimated_hours"] for week in implementation_tasks.values() for task_group in week.keys())

print(f"📅 Implementation Plan: {total_weeks} weeks, {total_tasks} tasks, {total_hours} hours")
print("=" * 60)

for week_name, week_tasks in implementation_tasks.items():
    week_num = week_name.split('_')[1]
    print(f"\n📅 WEEK {week_num.upper()}")
    
    for task_group, details in week_tasks.items():
        print(f"\n  🎯 {task_group.replace('_', ' ').title()} ({details['priority']})")
        print(f"     ⏱️ Estimated: {details['estimated_hours']} hours")
        for i, task in enumerate(details['tasks'], 1):
            print(f"     {i}. {task}")

print(f"\n🎯 Total Effort: {total_hours} hours = {total_hours/40:.1f} full-time weeks")

## 💡 7. Technical Considerations & Recommendations

### Performance Optimization
- **Async Processing**: All data collection via background jobs
- **Batch Processing**: Group anonymization in batches
- **Caching**: Redis caching for frequent queries
- **Connection Pooling**: PostgreSQL connection optimization

### Monitoring & Observability
- **Metrics**: Collection rate, processing lag, error rates
- **Alerts**: Failed jobs, high error rates, disk space
- **Dashboards**: Real-time collection stats, consent rates
- **Audit Logs**: All data access and processing events

### Scalability Planning
- **Horizontal Scaling**: Multiple worker instances
- **Database Sharding**: By time period if volume grows
- **CDN**: Static assets and API responses
- **Load Balancing**: Multiple FastAPI instances

In [ ]:
# Estimate volumes và capacity planning
import math

# Assumptions for volume estimation
volume_assumptions = {
    "daily_users": 100,        # Conservative start
    "growth_rate_monthly": 0.2, # 20% monthly growth
    "avg_questions_per_user": 50, # Across all questionnaires
    "consent_rate": 0.7,      # 70% opt-in for research
    "completion_rate": 0.8,   # 80% complete assessments
    "avg_event_size_bytes": 500, # JSON payload size
}

# Calculate monthly projections for 12 months
months = []
for month in range(1, 13):
    daily_users = volume_assumptions["daily_users"] * (1 + volume_assumptions["growth_rate_monthly"]) ** (month - 1)
    monthly_users = daily_users * 30
    research_participants = monthly_users * volume_assumptions["consent_rate"] * volume_assumptions["completion_rate"]
    monthly_events = research_participants * volume_assumptions["avg_questions_per_user"]
    data_size_mb = (monthly_events * volume_assumptions["avg_event_size_bytes"]) / 1024 / 1024
    
    months.append({
        "month": month,
        "daily_users": int(daily_users),
        "monthly_users": int(monthly_users),
        "research_participants": int(research_participants),
        "monthly_events": int(monthly_events),
        "data_size_mb": round(data_size_mb, 2)
    })

print("📈 Volume & Capacity Projections (12 months)")
print("=" * 50)
print(f"{'Month':<6} {'Daily Users':<12} {'Research Parts':<15} {'Events':<10} {'Data (MB)':<10}")
print("-" * 50)

for m in months[::3]:  # Show every 3 months
    print(f"{m['month']:<6} {m['daily_users']:<12} {m['research_participants']:<15} {m['monthly_events']:<10} {m['data_size_mb']:<10}")

# Year-end totals
year_end = months[-1]
total_year_data = sum(m['data_size_mb'] for m in months)
total_year_events = sum(m['monthly_events'] for m in months)

print(f"\n📊 Year-End Summary:")
print(f"   Peak Daily Users: {year_end['daily_users']:,}")
print(f"   Total Events: {total_year_events:,}")
print(f"   Total Data: {total_year_data:.1f} MB ({total_year_data/1024:.2f} GB)")

# Infrastructure recommendations
recommended_specs = {
    "database": f"{max(4, int(total_year_data/1024 * 3))} GB storage",
    "redis": f"{max(1, int(year_end['daily_users']/1000))} GB memory",
    "api_instances": max(2, int(year_end['daily_users']/500)),
    "worker_instances": max(2, int(year_end['monthly_events']/50000))
}

print(f"\n🖥️ Recommended Infrastructure (Year-End):")
for component, spec in recommended_specs.items():
    print(f"   {component.replace('_', ' ').title()}: {spec}")

## 🤝 8. Collaboration Plan & Next Steps

### Roles & Responsibilities
- **Backend Development**: FastAPI, Database, Workers
- **Frontend Integration**: Streamlit modifications, UI/UX
- **DevOps**: Docker, deployment, monitoring
- **Data Science**: Anonymization algorithms, research datasets
- **Compliance**: Legal review, privacy policies

### Decision Points Cần Thảo luận
1. **Technology Stack**: Confirm FastAPI + PostgreSQL + Redis
2. **Deployment Strategy**: Docker Compose vs Kubernetes
3. **Cloud Provider**: AWS vs GCP vs Azure (or on-premise)
4. **Monitoring Tools**: Prometheus/Grafana vs managed solutions
5. **Backup Strategy**: Frequency, retention, testing

### Quality Gates
- **Security Audit**: Before production deployment
- **Performance Testing**: Load testing with projected volumes
- **Compliance Review**: Legal and ethics review
- **User Acceptance**: Internal testing with consent flows

In [ ]:
# Tạo action items và decision matrix
immediate_decisions = {
    "high_priority": {
        "technology_confirmation": {
            "question": "Confirm tech stack: FastAPI + PostgreSQL + Redis?",
            "options": ["Approved as-is", "Modify components", "Alternative stack"],
            "impact": "HIGH - affects all development",
            "timeline": "This week"
        },
        "deployment_target": {
            "question": "Deployment environment preference?",
            "options": ["Local Docker", "Cloud (AWS/GCP)", "Hybrid"],
            "impact": "MEDIUM - affects infrastructure setup",
            "timeline": "Week 1"
        },
        "development_priority": {
            "question": "Which component to develop first?",
            "options": ["Database + API", "Consent UI", "Full integration"],
            "impact": "MEDIUM - affects development order",
            "timeline": "This week"
        }
    },
    "medium_priority": {
        "monitoring_approach": {
            "question": "Monitoring and observability strategy?",
            "options": ["Prometheus/Grafana", "Cloud native", "Simple logging"],
            "impact": "MEDIUM - affects operations",
            "timeline": "Week 2"
        },
        "backup_strategy": {
            "question": "Data backup and disaster recovery?",
            "options": ["Daily automated", "Real-time replication", "Manual backups"],
            "impact": "HIGH - affects data safety",
            "timeline": "Week 2"
        }
    }
}

print("🤝 Immediate Decisions Required")
print("=" * 35)

for priority, decisions in immediate_decisions.items():
    print(f"\n🚨 {priority.replace('_', ' ').title()} Decisions")
    
    for decision_id, details in decisions.items():
        print(f"\n  ❓ {details['question']}")
        print(f"     Timeline: {details['timeline']}")
        print(f"     Impact: {details['impact']}")
        print(f"     Options:")
        for i, option in enumerate(details['options'], 1):
            print(f"       {i}. {option}")

# Create action items
next_actions = [
    "✅ Review and approve architecture design",
    "✅ Confirm technology stack decisions", 
    "✅ Setup development environment",
    "✅ Create project repository structure",
    "✅ Define development milestones",
    "✅ Schedule weekly progress reviews",
    "✅ Begin Week 1 implementation tasks"
]

print(f"\n📋 Immediate Next Actions:")
for action in next_actions:
    print(f"  {action}")

print(f"\n🎯 Ready to proceed with implementation? Let's discuss the key decisions and move forward!")